# Introduction

Utiliser l'environnement _emigration_, cf. les instructions de la page d'accueil du dépôt et le fichier _environment.yml_

In [ ]:
import pprint, re, os, glob
from lxml import etree as etree
import pandas as pd
from matplotlib import pyplot as plt
from itertools import groupby
import sqlite3

### Liste des fichiers à traiter

* Définir les chemins des dossiers où se trouvent les fichiers
* Lister les fichiers du dossier XML

In [ ]:
### Chemin des documents en .docx
folder_path = 'fichiers/'
### Chemin du dossier des documents transformes 
folder_path_xml = 'fichiers_xml/'


In [ ]:
l_docs = []
for d in os.listdir(folder_path_xml):
    # vérifie que le document ne commence pas avec un '.', particularité MacOS pour docs ouverts
    if d[0] != '.':
        if('.html' in d):
            l_docs.append(d)



In [ ]:
len(l_docs), l_docs[:5]

## Exploration des documents produits

* transformer en XML les fichier HTML
* en cas d'erreur (problèmes d'encodage ou autre) une liste des fichiers qui posent problème est établie: variable _bad_files_ 
* il faut les ouvrir dans un éditeur de textes HTML/XML et les inspecter individuellement, puis corriger les erreurs directement dans le fichier word et refaire toute la procédure (afin de pouvoir continuer à travailler avec les originaux si on souhaite les enrichir)

In [ ]:
glob_list = [f for f in glob.glob(folder_path_xml + "*.html")]

In [ ]:
len(glob_list)

In [ ]:
glob_list[-3:]

In [ ]:
eparser = etree.ETCompatXMLParser(encoding='iso-8859-5')


bad_files =[]
ok_files = []
for f in glob_list:
    try:
        a = etree.parse(f,parser=eparser)
        ok_files.append(f)
    except Exception as exc :
        print(exc)
        bad_files.append(f)
        
        
        

In [ ]:
print(str(len(glob_list)) + ' : ' + str(len(ok_files)) + ' OK + ' + str(len(bad_files)) + ' not OK') 

In [ ]:
bad_files

# Exploration des _tags_

## Tags 

In [ ]:
tag_list = []
for doc in glob_list:
    doc_name = doc.replace(folder_path_xml, '')
    if doc not in bad_files:
        file = etree.parse(doc)

        ## //tei:*[contains("ptr name rs", name())]'
        """
        # Exemple:
        
        elements = file.xpath('//tei:name',
                              namespaces={'tei':'http://www.tei-c.org/ns/1.0'})
        """
        tags = file.xpath('//span[@class = "tags"]')
        i = 0
        for t in tags:
            if t.text : 
                if_1 = t.text 
            else: 
                if_1 = ''
            tag_list.append([t.text, doc_name])    
            ## print(str(i) + ' ' + t.tag + ' : ' + if_1 + ' (' + doc_name + ')')
            i += 1

In [ ]:
print(len(tag_list))
tag_list[89:105]

In [ ]:
splitted_tag_doc_list = []
for e in tag_list:
    if '/' in e[0]:
        splitted_tag_doc_list.append([e[0].split('/'), e[1]])
    else:
        splitted_tag_doc_list.append([[e[0]], e[1]])
#    print(b + ' : ' + a )

In [ ]:
splitted_tag_doc_list[89:105]

In [ ]:
### Chercher les codages avec plusieurs termes

termes_multiples = []
for t in splitted_tag_doc_list:
    if len(t[0]) > 1:
        termes_multiples.append([len(t[0]), t[0], t[1]])

In [ ]:
len(termes_multiples), termes_multiples[10:15]

In [ ]:
###  Créer un entrée par tag–document, 
# il y a donc moins de lignes que les tags encodées
# un tag par document, logique SQL: DISTINCT

tag_doc_list = []
tag_doc_list_distinct = []
for t in splitted_tag_doc_list:
    for e in t[0]:
        tag_doc_list.append([e, t[1]])
        if [e, t[1]] not in tag_doc_list_distinct:
            tag_doc_list_distinct.append([e, t[1]])
        else:
            pass

In [ ]:
### Liste complète avec éventuelle répétition dans le même document
tag_doc_list[89:100]

In [ ]:
### Si on souhaite ne retenir qu'un tag par document
tag_doc_list_distinct[50:60]

In [ ]:
### Liste des tags utilisés (en ordre alphabétique)
tl = sorted(list(set([e[0] for e in tag_doc_list])))
len(tl), tl[:5]

In [ ]:
### Afficher toute la liste
print(tl)

### Effectifs des tags utilisés

In [ ]:
### Liste complète des tags, répétitions dans les documents compris
# sin on ne souhaite pas de répétition utiliser la liste 'tag_doc_list_distinct'
all_tags = [t[0] for t in tag_doc_list]

In [ ]:
results = [[value, len(list(freq))] for value, freq in groupby(sorted(all_tags))]
print(results[:3])

In [ ]:
results.sort(key=lambda x:x[1], reverse=True) 
for r in results[:10]:
    print(r)

In [ ]:
### Transfromer la liste en data frame (Pandas)
df_results = pd.DataFrame(results, columns = ['tag','eff'])
df_results.head(3)

In [ ]:
### Afficher les mesure de centralité et dispersion des fréquences des tags utilisés
# on constate une très grande dispérsion, un classement serait utile en vue de l'analyse
df_results.describe().round(1)

In [ ]:
df_results = df_results.set_index('tag')
df_results.head(3)

In [ ]:
ax = df_results.sort_values(by='eff', ascending=True)\
        .plot(kind='barh',figsize=(15, 60), 
              color='#BDDAEC', width=0.90)
# plt.yticks(df_results.index,df_results['tag'])
#y = df_results.eff.value_counts(ascending=True)

eff = df_results.sort_values(by='eff', ascending=True).eff

for i, value in enumerate(eff):
        ax.text(1, i -.2, str(value), size=10)
plt.show()

# Tags and texts

In [ ]:
tags_texts_list = []
for doc in glob_list:  ### dans le premier texte   [:1]
    doc_name = doc.replace(folder_path_xml, '')
    if doc not in bad_files:
        file = etree.parse(doc)

        ## //tei:*[contains("ptr name rs", name())]'
        """
        elements = file.xpath('//tei:name',
                              namespaces={'tei':'http://www.tei-c.org/ns/1.0'})
        """
        spans = file.xpath('//span[@class = "semAnn"][./span/@class = "tags"]')
        print(len(spans))
        i = 0

        # while i < 5:
        for s in spans:
            tags = [e.text.split('/') for e in s.xpath('span[@class = "tags"]')]
            print(tags)
            for tag in tags[0]:
                print([i,tag, tags[0]])
                tags_texts_list.append([i, doc_name,  ", ".join(tags[0]), tag, "".join(s.xpath('./text()')) ])
            i += 1

In [ ]:
tags_texts_list[85:90]

In [ ]:
df_tags_texts_list = pd.DataFrame(tags_texts_list, columns=['id', 'document', 'tags_originaux', 'tag', 'texte'])

In [ ]:
chemin = 'resultats/tags_texts_list.csv'
df_tags_texts_list.to_csv(chemin, index=False)

#### Exporter dans table sqlite

In [ ]:
chemin = 'resultats/tags_and_texts.sqlite'
con = sqlite3.connect(chemin)

In [ ]:
### !!! Attention la table sera écrasée et réécrite entièremen à chaque exécution !!!
df_tags_texts_list.to_sql('tags_texts_list', con=con, if_exists='append')

In [ ]:
con.close()